In [3]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
# any number will do, as long as it is used consistently
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## STEP ONE: Gather the prepped data
We are going to get the data that the file data_wrangling outputted. This data is standardized, so the model will be able to predict y hat more accurately. 

Here, we are also going to split the prepped data into a train and dev set (it is called a test set here). 

In [4]:
prepped_data = pd.read_csv("prepped_data.csv")
prepped_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0,0,0,7541
1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0,0,-1,5506
2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0,0,1,2035
3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0,-1,0,3942
4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0,-1,-1,2914


In [9]:
prepped_x = prepped_data.values[:,:16]
prepped_y = prepped_data.values[:,16]
print(prepped_x.shape, prepped_y.shape)

(100046, 16) (100046,)


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(prepped_x, prepped_y, test_size=0.2, random_state=42)
X_train=X_train.astype("float64")
X_test=X_test.astype("float64")
y_train=y_train.astype("float64")
y_test=y_test.astype("float64")

## STEP TWO: Set up the parameters to test

In [5]:
n_estimators = [10, 50, 100, 250, 375, 500, 750, 1000]
max_features = ['auto', 'sqrt']
max_depth = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]# Create the random grid

param_distribs = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

## STEP THREE: the model
We will also perform a randomized search, so we can see which of the above parameters would perform the best. 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of ? rounds of training 
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=5, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(X_train, y_train)

Get the best model

In [ ]:
final_rf_model = rnd_search.best_estimator_
rnd_search.best_estimator_

## STEP FOUR: testing

In [ ]:
scores = cross_val_score(final_rf_model, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)